# Infer ***cellmask*** from a composite image -  2️⃣ 

--------------

## OVERVIEW
In this notebook, we will continue segmenting the different cell regions - the nucleus, cell, and cytoplasm - since they will be necessary for determining which organelle are in which cell. This is integral to our single cell analysis approach.

This notebook goes through the workflow steps to segment the ***cell*** from a composite image made from a combination of fluorescent organelle marker.

> WARNING: 🚨🚨🚨🚨
>> Because we do NOT have a direct cell membrane or cell-fill signal, this segmentation is difficult and potentially problematic. The cell mask will be used to determine the area of each single in our single cell analysis.



## OBJECTIVE: 
### ✅ Infer sub-cellular component #2: ***cellmask***
Segment the cell area -- the ***cellmask*** -- from a composite image of multiple organelle markers combined. This method is used in the case where there is no cell fill/membrane marker.

In this example, there are multiple cells per field of view, but some of the cells do not contain all of the fluorescent markers (due to the transfection protocol used). Therefore, we will use a single cell selection process to identify the cell with the highest total fluorescence after all markers are combined. This assumption could lead to the selection of the "wrong" cell, but in pratice it has worked reasonably well. *In the future, more sophisticated quality control methods could be implemented to select cells with all of the labels.*

The cellmask will be necessary for determining the area of individual cells in our single cell analysis approach.

> ***Biological relevance:*** 
> The combination of organelle markers used to create the composite image for the cellmask segmentation depends on the organelle labeles used and the cell type. In this example, the current selection includes the lysosomes, ER, and Golgi (e.g., Ch = 1, 3, 5) which have some intracellular background fluorescence - likely from off target marker localization. The lipid droplet channel (e.g., `ch = 6`) could also be included to produce a more unbiased selection of the entire cellmask as it binds to all cellular membranes to a small extent. However, the drawback of this is that it is present in every cell which makes downstream cell selection of a single cell more challenging. 
>
> *It is important to consider specifics of your system as the cell type and labeling method may differ from the example above.*



## IMPORTS

In [1]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict
from typing import Union, Tuple, List

import numpy as np

from scipy import ndimage as ndi
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_slice_by_slice )
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.utils import hole_filling

from skimage import filters
from skimage.segmentation import watershed, clear_border
from skimage.morphology import remove_small_holes   # function for post-processing (size filter)
from skimage.measure import label

# # package for io 
from aicsimageio import AICSImage

import napari

### import local python functions in ../infer_subc
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))


from infer_subc.core.file_io import (read_czi_image,
                                     read_ome_image,
                                     import_inferred_organelle,
                                     export_inferred_organelle,
                                     list_image_files)

                                             
from infer_subc.core.img import *
from infer_subc.organelles import (get_nuclei, 
                                   non_linear_cellmask_transform,
                                   choose_max_label_cellmask_union_nucleus)
from infer_subc.constants import (TEST_IMG_N,
                                  NUC_CH ,
                                  LYSO_CH ,
                                  MITO_CH ,
                                  GOLGI_CH ,
                                  PEROX_CH ,
                                  ER_CH ,
                                  LD_CH ,
                                  RESIDUAL_CH) 


%load_ext autoreload
%autoreload 2


## Get and load Image for processing

In [2]:
test_img_n = TEST_IMG_N

data_root_path = Path(os.path.expanduser("~")) / "Documents/Python Scripts/Infer-subc-2D"

in_data_path = data_root_path / "raw"
im_type = ".czi"

img_file_list = list_image_files(in_data_path,im_type)
test_img_name = img_file_list[test_img_n]

out_data_path = data_root_path / "out"
if not Path.exists(out_data_path):
    Path.mkdir(out_data_path)
    print(f"making {out_data_path}")

In [3]:
img_data,meta_dict = read_czi_image(test_img_name)

channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\ome_types\_convenience.py:112: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


----------
##  infer ***cellmask***

### summary of steps

➡️ INPUT (extract)
- segmented nuclei object (from [01_infer_nuclei](./02_infer_cellmask_from-composite.ipynb))
- create composite image from multiple organelle channels

PRE-PROCESSING
- rescale image intensities: 
    - min=0, max=1
- smooth image:
    - median filter (media size = user input)
    - gaussian filter (sigma = user input)
- log transform image
- apply scharr edge detection filter 
- combine log imge + scharr edge filtered intensity

CORE PROCESSING
- apply MO thresholding method from the Allen Cell [aicssegmentation](https://github.com/AllenCell/aics-segmentation) package

POST-PROCESSING
  - fill holes (hole size = user input)
  - remove small objects (object size = user input)

OUTPUT ➡️ 
  - label unique cell objects based on watershed seeded from the nuclei objects
  - select the single cell with the highest combined fluorescence 
  - save single ***cellmask*** (cell, CM) at unsigned integer 8-bit tif files


> ***Note:*** this pipeline will eventually include a selection step to identify the cellmask that are properly labeled with all fluorescent markers. This could be one single cell per image, or more if applicable data is available.


## GET nuclei segmentation output

In [4]:
# Import nuclei segmentation from output folder, or 
# run nuclei segmentation with hard coded segmentation settings (not ideal for most scenarios)
nuclei_labels = get_nuclei(img_data, meta_dict, out_data_path)

loaded  inferred 3D `nuclei`  from C:\Users\Shannon\Documents\Python Scripts\Infer-subc-2D\out 


In [5]:
# check the the image was correctly saved as uint16
nuclei_labels.dtype

dtype('uint16')

## EXTRACTION prototype

In [6]:
###################
# INPUT
###################
# add channels together with different weights
struct_img_raw = (6. * img_data[LYSO_CH].copy().astype(np.double) +
                  1. * img_data[ER_CH].copy().astype(np.double) + 
                  2. * img_data[GOLGI_CH].copy().astype(np.double))

In [7]:
# Creating a function to create composite image:
weights =  [0,6,0,2,0,1]
struct_img_raw2 = weighted_aggregate(img_data, *weights)

# use splat so we can also break out the arguments for our napari widget later
struct_img_raw3 = weighted_aggregate(img_data, 0,6,0,2,0,1)


# Comfirming the results are the same:
struct_img_raw[0,0:10,0], struct_img_raw2[0,0:10,0], struct_img_raw3[0,0:10,0]

(array([1386., 1045.,  760., 1137., 1011., 1808.,    0., 1538., 1783.,
        5187.]),
 array([1386., 1045.,  760., 1137., 1011., 1808.,    0., 1538., 1783.,
        5187.]),
 array([1386., 1045.,  760., 1137., 1011., 1808.,    0., 1538., 1783.,
        5187.]))

## PRE-PROCESSING prototype


In [8]:
###################
# PRE_PROCESSING
###################
med_filter_size = 15  
gaussian_smoothing_sigma = 1.34

structure_img_smooth = scale_and_smooth(struct_img_raw2,
                                        median_size = med_filter_size, 
                                        gauss_sigma = gaussian_smoothing_sigma)

In [9]:
# log scale the image, apply the scharr edge detection filter to logged image, add the two images together
composite_cellmask = non_linear_cellmask_transform(structure_img_smooth)

In [10]:
viewer = napari.Viewer()

26-May-23 17:49:52 - vispy    - WARNING  - QWindowsWindow::setGeometry: Unable to set geometry 1090x686+1040+172 (frame: 1108x733+1031+134) on QWidgetWindow/"_QtMainWindowClassWindow" on "\\.\DISPLAY1". Resulting geometry: 1365x859+1041+179 (frame: 1383x906+1032+141) margins: 9, 38, 9, 9 minimum size: 612x589 MINMAXINFO maxSize=0,0 maxpos=0,0 mintrack=630,636 maxtrack=0,0)
26-May-23 17:49:52 - vispy    - WARNING  - QWindowsWindow::setGeometry: Unable to set geometry 3844x1339+1041+179 (frame: 3860x1378+1033+148) on QWidgetWindow/"_QtMainWindowClassWindow" on "\\.\DISPLAY2". Resulting geometry: 3074x1070+1040+172 (frame: 3090x1109+1032+141) margins: 8, 31, 8, 8 minimum size: 612x587 MINMAXINFO maxSize=0,0 maxpos=0,0 mintrack=628,626 maxtrack=0,0)


In [11]:
viewer.add_image(structure_img_smooth)
viewer.add_image(composite_cellmask)

<Image layer 'composite_cellmask' at 0x18b62bf03a0>

## CORE PROCESSING prototype

In [12]:
###################
# CORE_PROCESSING
###################
# threshold the composite image after log/edge detection using the MO filter function from aicssegmentation - this applies a global threshold, then a local threshold to produce a semantic segmentation
thresh_method = 'ave'
cutoff_size =  150
thresh_adj = 0.5

bw = masked_object_thresh(composite_cellmask, 
                          global_method=thresh_method, 
                          cutoff_size=cutoff_size, 
                          local_adjust=thresh_adj)

In [13]:
viewer.add_image(bw)

<Image layer 'bw' at 0x18b8a4d76d0>

## POST-PROCESSING prototype

In [14]:
###################
# POST_PROCESSING
###################
hole_min_width = 0
hole_max_width = 50

small_object_width = 45

# removed_holes = hole_filling_linear_size(li_thresholded, 
#                                          hole_min=hole_min_width, 
#                                          hole_max=hole_max_width)
# 
# cleaned_img = size_filter_linear_size(removed_holes,
#                                       min_size=small_object_width)

cleaned_img2 = fill_and_filter_linear_size(bw, 
                                           hole_min=hole_min_width, 
                                           hole_max=hole_max_width, 
                                           min_size= small_object_width)

In [15]:
viewer.add_image(cleaned_img2)

<Image layer 'cleaned_img2' at 0x18b8a59d0c0>

## POST POST-PROCESSING prototype

In [19]:
###################
# POST- POST_PROCESSING
###################
# apply a watershed to the inverted image using the nuclei as a seed for each cell
watershed_method = '3D'
cellmask_labels = masked_inverted_watershed(structure_img_smooth, 
                                            nuclei_labels, 
                                            cleaned_img2,
                                            method=watershed_method)

# find the cell with the highest total fluorescence after combining all channels together
keep_label = get_max_label(composite_cellmask, 
                           cellmask_labels)


# combine the above and find the nucleus associated to the highest fluorescence cell
cellmask_out = choose_max_label_cellmask_union_nucleus(structure_img_smooth,
                                                       cleaned_img2, 
                                                       nuclei_labels,
                                                       watershed_method=watershed_method)

cellmask = cellmask_out.astype(bool)

In [20]:
cellmask.dtype

dtype('bool')

## LABELING prototype

No labeling steps needed for this workflow.

## Visualize with `napari` 1
Visualize the first-pass segmentation and labeling with `napari`.

In [ ]:
viewer = napari.view_image(cleaned_img2, 
                           scale=scale)

In [ ]:
viewer.add_labels(
    cellmask, 
    scale=scale)

## SAVE inferred nuclei to .tif file

Based on the _prototyping_ above define the function to infer nuclei.  


In [ ]:
out_file_n = export_inferred_organelle(cellmask, "cell", meta_dict, out_data_path)

----------
## DEFINE `infer_cellmask_fromcomposite` function

Based on the _prototyping_ above define the function to infer nuclei. 

> NOTE: these functions mainly serve for downstream prototyping in the notebooks. Each step above has an independent function that is implemented in the plugin for easy of use.

In [ ]:
##########################
# 1. infer_cellmask_fromaggr
##########################
def _infer_cellmask_fromcomposite(in_img: np.ndarray,
                                  weights: list[int],
                                  nuclei_labels: np.ndarray,
                                  median_sz: int,
                                  gauss_sig: float,
                                  mo_method: str,
                                  mo_adjust: float,
                                  mo_cutoff_size: int,
                                  min_hole_w: int,
                                  max_hole_w: int,
                                  small_obj_w: int,
                                  watershed_method: str
                                  ) -> np.ndarray:
    """
    Procedure to infer cellmask from linear unmixed input.

    Parameters
    ------------
    in_img: 
        a 3d image containing all the channels
    weights:
        a list of int that corresond to the weights for each channel in the composite; use 0 if a channel should not be included in the composite image
    nuclei_labels: 
        a 3d image containing the inferred nuclei labels
    median_sz: 
        width of median filter for _cellmask_ signal
    gauss_sig: 
        sigma for gaussian smoothing of _cellmask_ signal
    mo_method: 
         which method to use for calculating global threshold. Options include:
         "triangle" (or "tri"), "median" (or "med"), and "ave_tri_med" (or "ave").
         "ave" refers the average of "triangle" threshold and "mean" threshold.
    mo_adjust: 
        Masked Object threshold `local_adjust`
    mo_cutoff_size: 
        Masked Object threshold `size_min`
    max_hole_w: 
        hole filling cutoff for cellmask signal post-processing
    small_obj_w: 
        minimu object size cutoff for cellmask signal post-processing
    watershed_method:
        determines if the watershed should be run 'sice-by-slice' or in '3D' 

    Returns
    -------------
    cellmask_mask:
        a logical/labels object defining boundaries of cellmask

    """
    ###################
    # EXTRACT
    ###################
    struct_img = weighted_aggregate(in_img, *weights)

    ###################
    # PRE_PROCESSING
    ###################                         
    struct_img =  scale_and_smooth(struct_img,
                                   median_sz = median_sz, 
                                   gauss_sig = gauss_sig)
    

    struct_img_non_lin = non_linear_cellmask_transform(struct_img)

    ###################
    # CORE_PROCESSING
    ###################
    struct_obj = masked_object_thresh(struct_img_non_lin, 
                                      global_method=mo_method, 
                                      cutoff_size=mo_cutoff_size, 
                                      local_adjust=mo_adjust)               

    ###################
    # POST_PROCESSING
    ###################
    struct_obj = fill_and_filter_linear_size(struct_obj, 
                                             hole_min=min_hole_w, 
                                             hole_max=max_hole_w, 
                                             min_size= small_obj_w)

    ###################
    # POST- POST_PROCESSING
    ###################
    cellmask_out = choose_max_label_cellmask_union_nucleus(struct_img, 
                                                           struct_obj, 
                                                           nuclei_labels, 
                                                           watershed_method=watershed_method) 

    return cellmask_out.astype(bool)

## DEFINE `_fixed_infer_cellmask_fromcomposite` function

Based on the _prototyping_ above define the function to infer cellmask. with a *fixed* set of parameters for each step in the procedure.  That is they are all "hard coded"

In [ ]:
##########################
# 1. fixed_infer_cellmask_fromaggr
##########################
def _fixed_infer_cellmask_fromcomposite(in_img: np.ndarray, nuclei_labels: np.ndarray) -> np.ndarray:
    """
    Procedure to infer cellmask from linearly unmixed input, with a *fixed* set of parameters for each step in the procedure.  i.e. "hard coded"

    Parameters
    ------------
    in_img: 
        a 3d image containing all the channels
    nuclei_labels: 
        a 3d image containing the inferred nuclei

    Returns
    -------------
    cellmask_mask:
        a logical/labels object defining boundaries of cellmask
    """
    

    ###################
    # PARAMETERS
    ###################   
    weights = [0,6,0,2,0,1]
    median_sz = 15
    gauss_sig = 1.34
    mo_method = "ave"
    mo_adjust = 0.5
    mo_cutoff_size = 150
    hole_min_width = 0
    hole_max_width = 50
    small_obj_w = 45
    watershed_method = '3D'

    cellmask_out = _infer_cellmask_fromcomposite(in_img,
                                                weights,
                                                nuclei_labels,
                                                median_sz,
                                                gauss_sig,
                                                mo_method,
                                                mo_adjust,
                                                mo_cutoff_size,
                                                hole_min_width,
                                                hole_max_width,
                                                small_obj_w,
                                                watershed_method) 

    return cellmask_out

## TEST `_infer_cellmask_fromcomposite`  function defined above

In [ ]:
_CM_object =  _fixed_infer_cellmask_fromcomposite(img_data, nuclei_labels) 

_CM_object.dtype

In [ ]:
np.all(cellmask == _CM_object)

--------------------------

# TEST `infer_cellmask_fromcomposite` exported functions


In [ ]:
from infer_subc.organelles import fixed_infer_cellmask_fromcomposite

cellmask_mask =  fixed_infer_cellmask_fromcomposite(img_data, nuclei_labels) 

In [ ]:
np.all(cellmask_out == cellmask_mask)

## Visualize  2


In [ ]:
viewer.add_labels(
    cellmask_mask,
    scale=scale)

In [ ]:
from napari.utils.notebook_display import nbscreenshot

nbscreenshot(viewer, canvas_only=True)

In [ ]:
viewer.close()

-------------
## SUMMARY

The above details how the nuclei object is inferred.  

### NEXT: INFER CYTOPLASM

proceed to [03_infer_cytoplasm.ipynb](./03_infer_cytoplasm.ipynb)
